In [1]:
import numpy as np
from sktime.classification.kernel_based import RocketClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

In [2]:
from sktime.datasets import load_arrow_head  # univariate dataset
X_train, y_train = load_arrow_head(split="test", return_X_y=True)
X_test, y_test = load_arrow_head(split="train", return_X_y=True)
print(X_train.shape, X_test.shape)

X_train

(175, 1) (36, 1)


,dim_0
0,0 -1.907777 1 -1.904890 2 -1.88856...
1,0 -1.829855 1 -1.808497 2 -1.79577...
2,0 -1.801563 1 -1.775734 2 -1.72848...
3,0 -1.885718 1 -1.855170 2 -1.84519...
4,0 -1.959051 1 -1.974930 2 -1.97141...
...,...
170,0 -1.625142 1 -1.622988 2 -1.62606...
171,0 -1.657757 1 -1.664673 2 -1.63264...
172,0 -1.603279 1 -1.587365 2 -1.57740...
173,0 -1.739020 1 -1.741534 2 -1.73286...


In [3]:
from sktime.datatypes._panel._convert import (
    from_2d_array_to_nested,
    from_nested_to_2d_array,
    is_nested_dataframe,
)

x_np = from_nested_to_2d_array(X_train)
#x = x_np.iloc[:, :5] # Using first 5 steps for faster training
X_train = from_2d_array_to_nested(x_np)

X_train

,0
0,0 -1.907777 1 -1.904890 2 -1.88856...
1,0 -1.829855 1 -1.808497 2 -1.79577...
2,0 -1.801563 1 -1.775734 2 -1.72848...
3,0 -1.885718 1 -1.855170 2 -1.84519...
4,0 -1.959051 1 -1.974930 2 -1.97141...
...,...
170,0 -1.625142 1 -1.622988 2 -1.62606...
171,0 -1.657757 1 -1.664673 2 -1.63264...
172,0 -1.603279 1 -1.587365 2 -1.57740...
173,0 -1.739020 1 -1.741534 2 -1.73286...


In [4]:
# ECG dataset
# from sktime.datasets import load_from_arff_to_dataframe
# from IPython.display import display

# X_train, y_train = load_from_arff_to_dataframe('./ECG5000/ECG5000_TRAIN.arff')
# X_test, y_test = load_from_arff_to_dataframe('./ECG5000/ECG5000_TEST.arff')


In [5]:
rocket = RocketClassifier()
rocket.fit(X_train, y_train)
y_pred = rocket.predict(X_test)
accuracy_score(y_test, y_pred)

0.9166666666666666

In [6]:

X_pyreal = np.empty((X_train.shape[0], X_train.iloc[0][0].shape[0]))
print(X_pyreal.shape)
for i in range(X_train.shape[0]):
    for j in range(X_train.iloc[0][0].shape[0]):
        X_pyreal[i,j] = X_train.iloc[i][0][j]

X_pyreal = pd.DataFrame(X_pyreal)

(175, 251)


In [7]:
from sktime.datatypes._panel._convert import (
    from_2d_array_to_nested
)
class wrapped_model(object):
    def __init__(self, model):
        self.model = model

    def __call__(self, X):
        self.predict(X)

    def transform(self, data):
        df = from_2d_array_to_nested(data)
        return df
    
    def predict(self, X):
        X = self.transform(X)
        # without astype, the output is in <u1
        return self.model.predict(X).astype('int')

model = wrapped_model(rocket)
print(model.predict(X_pyreal))


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [8]:

from pyreal.explainers.time.shap_importance import MaskImportance

# change window_size to aggregate different timesteps
explainer = MaskImportance(model, X_pyreal, window_size=3)
explainer.fit()
explanation, X_interpret = explainer.produce(X_pyreal.iloc[:1, :])

print(explanation)


ImportError: cannot import name 'MaskImportance' from 'pyreal.explainers.time.shap_importance' (C:\Users\Ola\Documents\github\pyreal\pyreal\explainers\time\shap_importance.py)